<center><img src="../Picture Data/logo.png" alt="Header" style="width: 800px;"/></center>

@Copyright (C): 2010-2019, Shenzhen Yahboom Tech  
@Author: Malloy.Yuan  
@Date: 2019-07-17 10:10:02  
@LastEditors: Malloy.Yuan  
@LastEditTime: 2019-09-17 17:54:19  

# Object follow-Basic
In this journey, we will show how to track objects using JetBot!
We will use a pre-trained neural network trained on [COCO dataset] (http://cocodataset.org) to detect 90 different common objects. These include:

* Person (index 0)
* Cup (index 47)

We need to import the ``ObjectDetector`` class, which uses our pre-trained SSD engine. There are many other classes (you can check [this file]
(https://github.com/tensorflow/models/blob/master/research/object_detection/data/mscoco_complete_label_map.pbt)
Get a complete list of class indexes).

This model comes from [TensorFlow Object Detection API]
(https://github.com/tensorflow/models/tree/master/research/object_detection)

The API also provides utilities for object detector training for custom tasks. Once the model is trained, we use the NVIDIA TensorRT to optimize the Jetson Nano. This makes the network very fast and can control Jetbot in real time.

First, we need to import the "ObjectDetector" class, which uses our pre-trained SSD engine.

### Single camera image detection

In [1]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [1]:
from jetbot import ObjectDetector

model = ObjectDetector('ssd_mobilenet_v2_coco.engine')

Internally, the 'ObjectDetector' class uses the TensorRT Python API to execute the engine we provide. It is also responsible for pre-processing the input of the neural network and parsing the detected objects.

Currently, it only works with engines created with the ``jetbotssd_tensorrt`` package. This package has utilities for converting models from the TensorFlow object detection API to the optimized TensorRT engine. Next, let's initialize the camera. Our detector needs 300x300 pixels of input, so we will set this parameter when creating the camera.

In [2]:
from jetbot import Camera

#camera = Camera.instance(width=224, height=224, fps=10)
camera = Camera.instance(width=300, height=300)

Next, let's use some camera input to execute our network. By default, the ``ObjectDetector`` class expects the camera to generate a format of ``bgr8``.

However, if the input format is different, you can override the default preprocessor function.
If there are any COCO objects in the camera's field of view, they should now be stored in the ``detections`` variable, 

# Display detected objects in the text area

we print them out by the code shown below:

In [15]:
detections = model(camera.value)

print(detections)

[[{'label': 3, 'confidence': 0.7065955400466919, 'bbox': [0.7674810886383057, 0.46338123083114624, 0.8945862054824829, 0.5225367546081543]}, {'label': 8, 'confidence': 0.6001883745193481, 'bbox': [0.3486727178096771, 0.20762529969215393, 0.7051639556884766, 0.5552468299865723]}, {'label': 3, 'confidence': 0.5464961528778076, 'bbox': [0.35549741983413696, 0.20767933130264282, 0.7034186720848083, 0.5503236651420593]}, {'label': 10, 'confidence': 0.34554681181907654, 'bbox': [0.7413750886917114, 0.2913311719894409, 0.7752922773361206, 0.3802035450935364]}, {'label': 3, 'confidence': 0.32209932804107666, 'bbox': [0.2339305877685547, 0.4818982183933258, 0.25664815306663513, 0.5073009729385376]}]]


In [16]:
from IPython.display import display
import ipywidgets.widgets as widgets

detections_widget = widgets.Textarea()
detections_widget.value = str(detections)
display(detections_widget)

Textarea(value="[[{'label': 3, 'confidence': 0.7065955400466919, 'bbox': [0.7674810886383057, 0.46338123083114…

You can see the label, confidence, and border position of each object detected in each image. In this example there is only one image (our camera).

To print the first object detected in the first image, we can call the following command

>If no object detected, an error may be appear here.

In [14]:
image_number = 0
object_number = 0

print(detections[image_number][object_number])

{'label': 3, 'confidence': 0.8801805377006531, 'bbox': [0.0, 0.3776707649230957, 0.846367359161377, 0.8756875991821289]}


> If no object is detected, an error may be appear here.

# Control the robot to follow the center object

Now, we want the robot to follow the object of the specified class. To do this, we will do the following work

1. Detect objects that match the specified class
2. Select the object closest to the center of the camera's field of view. This is the target object.
3. Guide the robot to move to the target object, otherwise it will drift

We will also create widgets that control the target object label, robot speed and cornering gain, and control the speed of the robot's cornering based on the distance between the target object and the center of the robot's field of view.

In [8]:
import torch
import torchvision
import torch.nn.functional as F
import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    #Image zoomed to 224,224 versus 224,244 obstacle avoidance model
    x = cv2.resize(x, (224, 224))
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

Create a robot instance of the drive motor.

In [9]:
from jetbot import Robot

robot = Robot()

We display all control widgets and connect the network execution function to the camera update.

In [10]:
from jetbot import bgr8_to_jpeg

image_widget = widgets.Image(format='jpeg', width=300, height=300)
label_widget = widgets.IntText(value=1, description='tracked label')
speed_widget = widgets.FloatSlider(value=0.4, min=0.0, max=1.0, description='speed')
turn_gain_widget = widgets.FloatSlider(value=0.8, min=0.0, max=2.0, description='turn gain')

display(widgets.VBox([
    widgets.HBox([image_widget]),
    label_widget,
    speed_widget,
    turn_gain_widget
]))

width = int(image_widget.width)
height = int(image_widget.height)

def detection_center(detection):
    """Calculate the center x, y coordinates of the object"""
    bbox = detection['bbox']
    center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
    center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
    return (center_x, center_y)
    
def norm(vec):
    """Calculate the length of a two-dimensional vector"""
    return np.sqrt(vec[0]**2 + vec[1]**2)

def closest_detection(detections):
    """Find the detection closest to the center of the image"""
    closest_detection = None
    for det in detections:
        center = detection_center(det)
        if closest_detection is None:
            closest_detection = det
        elif norm(detection_center(det)) < norm(detection_center(closest_detection)):
            closest_detection = det
    return closest_detection
        
def execute(change):
    image = change['new']
    # Calculate all detected objects
    detections = model(image)
    
    # Draw all detected objects on the image
    for det in detections[0]:
        bbox = det['bbox']
        cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (255, 0, 0), 2)
    
    # Select to match the detection of the selected class label
    matching_detections = [d for d in detections[0] if d['label'] == int(label_widget.value)]
    
    # Let the detection be closest to the center of the field and draw it
    det = closest_detection(matching_detections)
    if det is not None:
        bbox = det['bbox']
        cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (0, 255, 0), 5)
    
    # If don't detected object,jetbot car will keep advance
    if det is None:
        pass
        robot.forward(0)
        
    # If detected object,control Jetbot follow object
    else:
        # Move the robot forward and control the x distance between the proportional target and the center
        center = detection_center(det)
        robot.set_motors(
            float(speed_widget.value + turn_gain_widget.value * center[0]),
            float(speed_widget.value - turn_gain_widget.value * center[0])
        )
    
    # Update image display to widget
    image_widget.value = bgr8_to_jpeg(image)
    
execute({'new': camera.value})

Because the detection function is called through the screen change before, the delay is relatively large, resulting in a poor following effect. We are temporarily using a loop to continuously call the detection function to improve the smoothness of the picture and improve the recognition effect.

In [11]:
#camera.unobserve_all()
#camera.observe(execute, names='value')

import threading
import inspect
import ctypes
''' definition of the close thread function'''
def _async_raise(tid, exctype):
  """raises the exception, performs cleanup if needed"""
  tid = ctypes.c_long(tid)
  if not inspect.isclass(exctype):
    exctype = type(exctype)
  res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
  if res == 0:
    raise ValueError("invalid thread id")
  elif res != 1:
    # """if it returns a number greater than one, you're in trouble,
    # and you should call it again with exc=NULL to revert the effect"""
    ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
    raise SystemError("PyThreadState_SetAsyncExc failed")
def stop_thread(thread):
  _async_raise(thread.ident, SystemExit)
'''The function of thread 1, which continuously calls the detection function'''
def test():
    while True:
        execute({'new': camera.value})        

thread1 = threading.Thread(target=test)
thread1.setDaemon(False)
thread1.start()


If the robot is not blocked, you can see that the blue box surrounds the detected object and the target object (the object that the robot follows) will be displayed in green.

When the target is discovered, the robot should turn to the target.

You can call the following code block to manually disconnect the camera and stop the robot.

In [ ]:
import time
stop_thread(thread1)
camera.unobserve_all()
time.sleep(1.0)
robot.stop()